In [1]:
import pandas as pd

In [9]:
%%time
df = pd.read_csv("hand_combinations.csv", index_col=0).astype('int8')
df.rename(index=lambda i: i.replace("'", ""), inplace=True)
print(f"Memory usage: {round(sum(df.memory_usage(deep=True)) / 1024 / 1024, 1)}MB\n")
df

Memory usage: 433.2MB

CPU times: total: 15.2 s
Wall time: 15.2 s


,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
"(♠A, ♠2, ♠3, ♠4, ♠5)",0,1,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
"(♠A, ♠2, ♠3, ♠4, ♠6)",0,1,1,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
"(♠A, ♠2, ♠3, ♠4, ♠7)",0,1,1,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
"(♠A, ♠2, ♠3, ♠4, ♠8)",0,1,1,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
"(♠A, ♠2, ♠3, ♠4, ♠9)",0,1,1,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(♦8, ♦9, ♦10, ♦J, ♦K)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,1,0
"(♦8, ♦9, ♦10, ♦Q, ♦K)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,1
"(♦8, ♦9, ♦J, ♦Q, ♦K)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,1,1
"(♦8, ♦10, ♦J, ♦Q, ♦K)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,1
